# Check out structured data for AHRQ-ML

In [1]:
import pyodbc
import pandas as pd  # sure takes a long time

In [2]:
conn = pyodbc.connect("DRIVER={SQL Server};SERVER=VHACDWRB03;DATABASE=ORD_Singh_201911038D")

In [3]:
result_df = pd.read_sql(sql="select * from information_schema.tables where table_name like '%B00%'", con=conn)

In [4]:
result_df

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_01_04_cohort,BASE TABLE
1,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_04_04_Lab,BASE TABLE
2,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_05_04_Rad,BASE TABLE
3,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_06_Med_04_RxOutpat,BASE TABLE
4,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_06_Med_05_BCMA_Dispense...,BASE TABLE
5,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_06_Med_07_NonVAMed,BASE TABLE
6,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_06_Med_08_AllMed,BASE TABLE
7,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_08_Consult,BASE TABLE
8,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_12_ICD,BASE TABLE
9,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_14_Vital,BASE TABLE


# To do

- come back to "Rad" which is time of result but also type of test
- list of important meds


# Important tables

- RxOutpat: Very much recommend start with **selected** meds, not *all* meds.
- consult: again **selected** and presence/absence
- ICD

## lower or medium tier (not saying which is which)

- bcma
- non va meds
- lab
- vitals
- health factors
- note


In [9]:
pd.read_sql(sql="select count(*) from dflt._B00_ML4TrgPos_Y201621_01_04_cohort", con=conn)

,
0,1126


Cohort is one row per encounter. These are the patients for whom trigger is positive.

In [11]:
cohort_df = pd.read_sql(sql="select * from dflt._B00_ML4TrgPos_Y201621_01_04_cohort", con=conn)

In [12]:
cohort_df.shape

(1126, 12)

In [13]:
cohort_df.columns

Index(['TriggerType', 'patientSSN', 'EDStartDateTime', 'EDEndDateTime',
       'EDVisitReason', 'AdmitDateTime', 'B', 'outpatVisitSID', 'inpatientsid',
       'patientsid', 'sta3n', 'studyID'],
      dtype='object')

In [16]:
cohort_df.EDVisitReason.value_counts()

abd pain                                       92
GI                                             54
abdominal pain                                 32
ABD PAIN                                       22
dizziness                                      19
                                               ..
s                                               1
r/o SBO; nausea/emesis                          1
20/GI - TACHYCARDIA                             1
abd distention                                  1
Weak,confusion,falling episode d/t weakness     1
Name: EDVisitReason, Length: 691, dtype: int64

In [17]:
cohort_df.TriggerType.value_counts()

AbdominalPain    921
Dizziness        205
Name: TriggerType, dtype: int64

In [18]:
cohort_df.EDVisitReason.value_counts().shape

(691,)

In [75]:
def all_tables_from_str(sql_pattern, con=conn):
    """NOT FULLY TESTED.
    Input a pattern like %_b00%. Create a dict of data frames, 1 k:v for each SQL table that matches.
    """
    info_schema_df = pd.read_sql(sql="select * from information_schema.tables where table_name like '%s'" % sql_pattern, con=conn)
    
    tables = {}
    for tname in info_schema_df.TABLE_NAME:
        query_str = "select * from  " + str("Dflt.")+ str(tname)
        table_df = pd.read_sql(sql=query_str,con=conn)
        tables[tname.split('_')[-1]] = table_df
    return tables

In [74]:
# a00_d = all_tables_from_str('%B00%')  # takes a long time. Because big tables and select * .

## Copied out of Devika notebook

Trying to map a text string (key) to a few categories (values).

In [20]:
chief_c_dict = {'headache':'headache','dizziness':'dizziness','neuro':'neuro','ha':'headache','dizzy':'dizziness','migraine':'migraine',
        'fall':'fall','headaches':'headache','near syncope':'near syncope','abnormal ekg':'abnormal ekg'}

In [22]:
set(chief_c_dict.values())

{'abnormal ekg',
 'dizziness',
 'fall',
 'headache',
 'migraine',
 'near syncope',
 'neuro'}

# Explore just the 3 tables Andy thinks are hi value

In [27]:
consult = pd.read_sql(sql="select * from dflt._B00_ML4TrgPos_Y201621_08_Consult", con=conn)
rxoutpat = pd.read_sql(sql="select * from dflt._B00_ML4TrgPos_Y201621_06_Med_04_RxOutpat", con=conn)
icd = pd.read_sql(sql="select * from dflt._B00_ML4TrgPos_Y201621_12_ICD", con=conn)

These are big tables, apparently. `cohort_df` alreadly loaded in.

In [28]:
consult.shape

(146468, 11)

In [29]:
rxoutpat.shape

(404950, 13)

In [30]:
icd.shape

(865268, 5)

In [33]:
print(consult.columns)
print(rxoutpat.columns)
print(icd.columns)

Index(['PatientSSN', 'Sta3n', 'PatientSID', 'requestDateTime', 'requestType',
       'ConStopCode', 'ConStopCodeName', 'ToRequestServiceName',
       'CPRSOrderResultsDateTime', 'ConsultSID', 'ToRequestserviceSID'],
      dtype='object')
Index(['PatientSSN', 'DrugClassCode', 'LocalDrugNameWithDose', 'NDC',
       'DispensedDate', 'Qty', 'QtyNumeric', 'DaysSupply', 'Sta3n',
       'PatientSID', 'RxOutpatSID', 'LocalDrugSID', 'RxOutpatFillSID'],
      dtype='object')
Index(['PatientSSN', 'DiagDateTime', 'ICD', 'ICDDiagnosis', 'IsPrincipalDiag'], dtype='object')


In [39]:
cohort_df.columns

Index(['TriggerType', 'patientSSN', 'EDStartDateTime', 'EDEndDateTime',
       'EDVisitReason', 'AdmitDateTime', 'B', 'outpatVisitSID', 'inpatientsid',
       'patientsid', 'sta3n', 'studyID'],
      dtype='object')

# We will start with the ICD table

In [65]:
myssn = '000000000'  # insert real ssn here. Removed because PHI.

Just for demo purposes, print out *all* ICD, any date, but sorted by date. Observe that some are from "future" (after index date).

In [66]:
icd.DiagDateTime = pd.to_datetime(icd.DiagDateTime)  # modify in place
icd[icd.PatientSSN == myssn].sort_values(by = 'DiagDateTime')

,PatientSSN,DiagDateTime,ICD,ICDDiagnosis,IsPrincipalDiag


Now look at cohort table. We deliberately selected a patient who happened to have >1 encounter in cohort table.

In [67]:
cohort_df[cohort_df.patientSSN == myssn]  # lower case patientSSN not PatientSSN

,TriggerType,patientSSN,EDStartDateTime,EDEndDateTime,EDVisitReason,AdmitDateTime,B,outpatVisitSID,inpatientsid,patientsid,sta3n,studyID


In [70]:
one_pt_icd = icd[icd.PatientSSN == myssn]

In [71]:
idx_date_str = '2000-01-01'  # insert real date. Removed because PHI.

## ML mustn't cheat: Use only the ICDs BEFORE index date

In [72]:
past_icd = one_pt_icd[one_pt_icd.DiagDateTime < pd.to_datetime(idx_date_str)]

very likely ok to ignore `IsPrincipalDiag` column.

## Final step: All you care about is the count of ICDs

Those can maybe be some of your features? Depending on how well your method likes this dimensionality.

In [73]:
past_icd.ICDDiagnosis.value_counts()  # warning. assumes 1:1 ICD:string

Series([], Name: ICDDiagnosis, dtype: int64)

# Reminder of Important tables

- RxOutpat: Very much recommend start with **selected** meds, not *all* meds.
- consult: again **selected** and presence/absence
- ICD

# RXOutpat table

We won't go in detail yet. But use RxOutpat. Use the VA drug class column. Use date filled & days supply. First approximation: ignore dose. The person needs to be "on" the drug. Kind of that means "recently filled with a reasonable *N* of days supply." So like if *K* days supply, maybe he filled it *1.5 K* days ago (before index date) and no further??

That number 1.5 is entirely made up by me. There is no one number. Maybe *2 K* at most? I spent several years reading papers about this.

What does "selected" meds mean? Meds "obviously" related to stroke, at least to a doctor:

- aspirin
- anything in VA drug class 'anticoagulant': warfarin, Coumadin, apixaban, anything ending in "*xaban"
- clopidogrel / Plavix
- anything ending "*statin"

Meds with lots of them in the class (related to stroke risk but messier):

- Lots of anti-hypertensives. Lots.

# Where are the labels?

Good question! Here, for example:

`P:\ORD_Singh_201911038D\Ashish\HighRiskAbdominalPain\AbdominalPain-Validation review`

They live in MS Access files. Some will be under `Ashish\HighRiskDizz......` etc.

**Viral** will manage the export. ML folks: remember not to edit any MS Access contsnts.